# Example: Binomial Lattice Models of Equity Share Price

## Background
A binomial lattice model assumes that each discrete time increment, the state of the system, e.g., the share price of equity, the short rate, etc., can either increase by a factor $u$ with probability $p$ or decrease by a factor $d$ with probability $(1-p)$. 

<div>
    <center>
        <img src="figs/Fig-Binomial-Lattice-Schematic.png" width="300"/>
    </center>
</div>

Different models can be developed to compute the tuple $(u,d,p)$. However, for now, let's use historical data to learn typical values governing the price movement. Thus, we’ll take an approach similar to the short-rate calculations discussed previously. 

## Learning Objectives
The objective of this example is to familiarize students with the application of simple probability mass functions to predict future uncertain quantities such as commodity prices, equity share prices, interest rates, etc. In particular, in this example, we will:

* __Objective 1__: Estimate typical values for the tuple $(u,d,p)$ using historical data for different the share price of different firms.  
* __Objective 2__: Compare bionomial lattice model simulations of share price, and statistical properties of the return, versus historical data 

## Setup
We set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our example problem.

### Packages
Fill me in.

### Types
Fill me in.

### Functions
Fill me in.

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-132/module-1`


## Objective 1: Estimate values of the up, down and probability of an up move lattice parameters
To create a binomial lattice model for future share prices, we need to gather data on three parameters: $p$, $u$, and $d$:

* The $p$ parameter denotes the probability that the share price will rise, i.e., an `up` move between two time periods $j\rightarrow{j+1}$. A binary lattice model only allow `up` and `down` moves, thus, the probability of a `down` move is $1-p$.
* The $u$ parameter denotes the amount of an `up` move, i.e., if $S_{j}$ denotes the share price in period $j$, and $S_{j+1}$ denotes the share price in the next period, then an `up` move gives $S_{j+1} = u\cdot{S}_{j}$. 
* The $d$ parameter denotes the amount of a `down` move, i.e., if $S_{j}$ denotes the share price in period $j$, and $S_{j+1}$ denotes the share price in the next period, then a `down` moves gives $S_{j+1} = d\cdot{S}_{j}$.

To esimate the $(u,d,p)$ parameters, let's load historical daily price data sets for firms that make up the [The Standard and Poor's 500, or simply the S&P 500](https://en.wikipedia.org/wiki/S%26P_500) which we've numbered as `Firm-1,...` for years `2018` through `2022` using the `loaddatafile(...)` function.

In [2]:
years = ["2018", "2019", "2020", "2021", "2022"];
firm = 1
dataset = Dict{String,DataFrame}();
for year ∈ years
    dataset[year] = loaddatafile(firm=firm, year=year);
end

Next, combine all the `DataFrame` instances for each year into a single `DataFrame` holding data for all the years using [append!](https://docs.juliahub.com/DataFrames/AR9oZ/0.21.5/lib/functions/#Base.append!) function. Stored the combined `DataFrame` in the `df` variable:

In [3]:
df = dataset["2018"];
number_of_years = length(years);
for i ∈ 2:number_of_years
    year = years[i];
    df_next = dataset[year];
    append!(df,df_next)
end

Next, we need to calculate the number of `up` and `down` moves, and the magnitude of these moves. To do this, let's assume a share price model of the form:

$$
S_{j} = \exp\left(\mu_{j,j-1}\Delta{t}\right)\cdot{S_{j-1}}
$$

where $\mu_{j,j-1}$ denotes the _return_ (units: 1/time) and $\Delta{t}$ (units: time) denotes the length of time between time period $j-1\rightarrow{j}$. Solving for the return value $\mu_{j,j-1}$ gives the expression:

$$
\mu_{j,j-1} = \left(\frac{1}{\Delta{t}}\right)\cdot\ln\left(\frac{S_{j}}{S_{j-1}}\right)
$$

In [4]:
Δt = (1.0/365.0);
number_of_trading_days = nrow(df);
return_array = Array{Float64,1}(undef, number_of_trading_days-1)
for i ∈ 2:number_of_trading_days
    
    S₁ = df[i-1,:close];
    S₂ = df[i,:close];
    μ = (1/Δt)*log(S₂/S₁);
    return_array[i-1] = μ
end

Using the `return_array`, compute the number of up moves which is indicated by $\mu_{j,j-1}>0$ and estimate the probability of an `up` move `p`:

In [5]:
N₊ = findall(x->x>0,return_array) |> length
p = N₊/length(return_array)
println("The probability of an up move p = $(p)")

The probability of an up move p = 0.5147410358565737


Finally, let's compute the average magnitide of an `up` move: 

In [9]:
index_up_moves = findall(x->x>0, return_array);
uarray = Array{Float64,1}();
for index ∈ index_up_moves
    μ = return_array[index];
    u = exp(μ*Δt);
    push!(uarray,u)
end

ū = mean(uarray)
println("The mean magnitude of an up move ū = $(ū)")

The mean magnitude of an up move ū = 1.011058734867527


and the average magnitude of a `down` move:

In [11]:
index_down_moves = findall(x->x<0, return_array);
darray = Array{Float64,1}();
for index ∈ index_down_moves
    μ = return_array[index];
    d = exp(μ*Δt);
    push!(darray,d)
end

d̄ = mean(darray)
println("The mean magnitude of a down move d̄ = $(d̄)")

The mean magnitude of a down move d̄ = 0.9874046686280384


### Build binomial lattice model using historial $(u,d,p)$ parameters
Now that 

## Objective 2: Compare simulations from the Lattice Model with historical price data and stylized facts

In [7]:
number_of_trading_days

1256